<a href="https://colab.research.google.com/github/imji0319/PDSH/blob/master/nyc_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My Drive/data/nyc_split

/content/gdrive/My Drive/data/nyc_split


In [3]:
pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (0.13.0)


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
import lightgbm as lgbm

In [0]:
params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': -1,
        'verbose_eval':50,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'subsample_freq': 1,
        'colsample_bytree': 0.6,
        'reg_aplha': 1,
        'reg_lambda': 0.001,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1 ,
        'metric':'rmse'
        
    }

In [0]:
path = 'nyc_0.feather'
df_train =pd.read_feather(path)
df_train = df_train.drop(columns=['level_0', 'index'])

In [0]:
y = df_train['fare_amount']
train = df_train.drop(columns=['fare_amount'])

X_train, X_test, y_train, y_test = train_test_split(train, y, random_state = 100, test_size=0.25)
del train, y

train_set = lgbm.Dataset(X_train, label=y_train) 
val_T = lgbm.Dataset(X_test, label=y_test)

del X_train, X_test, y_train, y_test





In [9]:
%%time
model = lgbm.train(params, train_set = train_set, num_boost_round=500,valid_sets=val_T, verbose_eval=50, early_stopping_rounds=100)


Training until validation scores don't improve for 100 rounds.
[50]	valid_0's rmse: 3.88596
[100]	valid_0's rmse: 3.5566
[150]	valid_0's rmse: 3.49362
[200]	valid_0's rmse: 3.45773
[250]	valid_0's rmse: 3.43192
[300]	valid_0's rmse: 3.4104
[350]	valid_0's rmse: 3.39651
[400]	valid_0's rmse: 3.38461
[450]	valid_0's rmse: 3.37479
[500]	valid_0's rmse: 3.36572
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 3.36572
CPU times: user 2min 20s, sys: 533 ms, total: 2min 21s
Wall time: 1min 11s


In [0]:
from lightgbm import Booster

In [11]:
model.save_model('lgbm_model.txt')

In [0]:
bst = lgbm.Booster(model_file='lgbm_model.txt')

In [13]:
bst

In [26]:
cd /content/gdrive/My Drive/data

/content/gdrive/My Drive/data


In [0]:
df_test = pd.read_csv('cleaning_test.csv')

In [0]:
x_pred = df_test.drop(columns=['key', 'pickup_datetime'])

In [0]:
prediction = bst.predict(x_pred, num_iteration = model.best_iteration)

In [18]:
submission = pd.DataFrame({'key' : df_test.key,
                          'fare_amount': prediction},
                         columns =['key','fare_amount'])

submission.head(10)

,key,fare_amount
0,2015-01-27 13:08:24.0000002,11.074502
1,2015-01-27 13:08:24.0000003,11.157420
2,2011-10-08 11:53:44.0000002,4.419889
3,2012-12-01 21:12:12.0000002,7.745060
4,2012-12-01 21:12:12.0000003,13.960660
5,2012-12-01 21:12:12.0000005,9.820040
6,2011-10-06 12:10:20.0000001,4.976940
7,2011-10-06 12:10:20.0000003,48.973880
8,2011-10-06 12:10:20.0000002,11.258480
9,2014-02-18 15:22:20.0000002,6.971797


In [0]:
submission.to_csv('lgbm_submission_expect_index.csv', index=False)

# loading model to train

In [6]:
cd /content/gdrive/My Drive/data/nyc_split

/content/gdrive/My Drive/data/nyc_split


In [14]:
# 분할된 데이터 로드 및 train 

%%time
for i in range(1,21):
    path = 'nyc_'+str(i)+'.feather'
    df_train =pd.read_feather(path)
    df_train = df_train.drop(columns=['level_0', 'index'])
    y = df_train['fare_amount']
    train = df_train.drop(columns=['fare_amount'])


    X_train, X_test, y_train, y_test = train_test_split(train, y, random_state = 100, test_size=0.25)
    del train, y

    train_set = lgbm.Dataset(X_train, label=y_train) 
    val_T = lgbm.Dataset(X_test, label=y_test)

    del X_train, X_test, y_train, y_test
    
    bst = lgbm.Booster(model_file='lgbm_model.txt')

    
    model = lgbm.train(params, train_set = train_set, num_boost_round=500,valid_sets=val_T, 
                                    verbose_eval=50, early_stopping_rounds=100, init_model=bst)
    
    del bst
    
    print(str(i) + '번 파일데이터가 트레이닝 되었습니다.')
    
    model.save_model('lgbm_model.txt')
    del model
    

Training until validation scores don't improve for 100 rounds.
[550]	valid_0's rmse: 3.50891
[600]	valid_0's rmse: 3.50149
[650]	valid_0's rmse: 3.49707
[700]	valid_0's rmse: 3.49231
[750]	valid_0's rmse: 3.48484
[800]	valid_0's rmse: 3.48091
[850]	valid_0's rmse: 3.47524
[900]	valid_0's rmse: 3.47087
[950]	valid_0's rmse: 3.46676
[1000]	valid_0's rmse: 3.46277
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 3.46277
1번 파일데이터가 트레이닝 되었습니다.
Training until validation scores don't improve for 100 rounds.
[1050]	valid_0's rmse: 3.24113
[1100]	valid_0's rmse: 3.2378
[1150]	valid_0's rmse: 3.23355
[1200]	valid_0's rmse: 3.23189
[1250]	valid_0's rmse: 3.22975
[1300]	valid_0's rmse: 3.22728
[1350]	valid_0's rmse: 3.2262
[1400]	valid_0's rmse: 3.22468
[1450]	valid_0's rmse: 3.22201
[1500]	valid_0's rmse: 3.2202
Did not meet early stopping. Best iteration is:
[1493]	valid_0's rmse: 3.22012
2번 파일데이터가 트레이닝 되었습니다.
Training until validation scores don't improve for 100 rounds.
[

In [0]:
model = lgbm.Booster(model_file='lgbm_model.txt')

In [8]:
cd /content/gdrive/My Drive/data

/content/gdrive/My Drive/data


In [0]:
df_test = pd.read_csv('cleaning_test.csv')
x_pred = df_test.drop(columns=['key', 'pickup_datetime'])

In [0]:
prediction = model.predict(x_pred, num_iteration = model.best_iteration)

In [11]:
submission = pd.DataFrame({'key' : df_test.key,
                          'fare_amount': prediction},
                         columns =['key','fare_amount'])

submission.head(10)

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.831969
1,2015-01-27 13:08:24.0000003,11.398784
2,2011-10-08 11:53:44.0000002,4.246675
3,2012-12-01 21:12:12.0000002,7.811351
4,2012-12-01 21:12:12.0000003,13.665502
5,2012-12-01 21:12:12.0000005,9.125567
6,2011-10-06 12:10:20.0000001,4.999745
7,2011-10-06 12:10:20.0000003,49.423231
8,2011-10-06 12:10:20.0000002,11.461672
9,2014-02-18 15:22:20.0000002,7.186082


In [0]:
submission.to_csv('lgbm_submission_all.csv', index=False)